
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
Root = "/content/drive/MyDrive/Colab Notebooks/RAVDESS_Emotional_speech_audio"
os.chdir(Root)

In [ ]:
ls

modelForPrediction1.sav  speech-emotion-recognition-ravdess-data/


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

array([[-5.00791290e+02,  3.33641396e+01, -2.19598770e+01, ...,
         4.66571539e-04,  3.32369033e-04,  1.58667855e-04],
       [-3.68171265e+02,  1.21564112e+01, -4.14419365e+01, ...,
         5.45278145e-03,  3.56062246e-03,  2.32826290e-03],
       [-6.60783386e+02,  5.15672340e+01,  8.21943188e+00, ...,
         3.79883044e-04,  1.51239467e-04,  8.75224796e-05],
       ...,
       [-6.38802979e+02,  4.69577484e+01,  4.58249189e-02, ...,
         1.31693814e-04,  1.22965052e-04,  8.94540499e-05],
       [-5.81731018e+02,  3.85710831e+01, -1.54516716e+01, ...,
         6.08591654e-05,  4.53564971e-05,  4.73912551e-05],
       [-6.41207336e+02,  4.49728851e+01, -1.83886886e+00, ...,
         3.89262023e-05,  3.05255780e-05,  2.94166712e-05]])

In [ ]:
y_train

['happy',
 'fearful',
 'disgust',
 'calm',
 'calm',
 'disgust',
 'calm',
 'disgust',
 'calm',
 'happy',
 'calm',
 'happy',
 'happy',
 'happy',
 'calm',
 'disgust',
 'happy',
 'happy',
 'calm',
 'fearful',
 'happy',
 'happy',
 'disgust',
 'happy',
 'disgust',
 'happy',
 'calm',
 'disgust',
 'fearful',
 'calm',
 'disgust',
 'disgust',
 'happy',
 'fearful',
 'calm',
 'fearful',
 'fearful',
 'disgust',
 'disgust',
 'fearful',
 'disgust',
 'disgust',
 'fearful',
 'fearful',
 'calm',
 'disgust',
 'happy',
 'disgust',
 'happy',
 'calm',
 'disgust',
 'disgust',
 'calm',
 'happy',
 'fearful',
 'happy',
 'disgust',
 'happy',
 'disgust',
 'happy',
 'disgust',
 'disgust',
 'happy',
 'disgust',
 'calm',
 'happy',
 'happy',
 'happy',
 'disgust',
 'disgust',
 'fearful',
 'happy',
 'fearful',
 'calm',
 'disgust',
 'calm',
 'calm',
 'happy',
 'fearful',
 'calm',
 'calm',
 'fearful',
 'disgust',
 'disgust',
 'disgust',
 'happy',
 'fearful',
 'fearful',
 'disgust',
 'fearful',
 'disgust',
 'happy',
 'hap

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(583, 195)


In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['disgust', 'fearful', 'calm', 'calm', 'calm', 'calm', 'calm',
       'disgust', 'fearful', 'fearful', 'fearful', 'calm', 'happy',
       'fearful', 'calm', 'calm', 'calm', 'disgust', 'disgust', 'fearful',
       'disgust', 'calm', 'calm', 'calm', 'fearful', 'fearful', 'calm',
       'fearful', 'calm', 'fearful', 'calm', 'calm', 'happy', 'disgust',
       'calm', 'calm', 'disgust', 'calm', 'calm', 'happy', 'calm',
       'disgust', 'calm', 'fearful', 'calm', 'fearful', 'happy', 'calm',
       'calm', 'calm', 'calm', 'happy', 'fearful', 'calm', 'calm',
       'fearful', 'happy', 'happy', 'calm', 'disgust', 'fearful',
       'fearful', 'disgust', 'disgust', 'disgust', 'fearful', 'fearful',
       'disgust', 'happy', 'calm', 'disgust', 'disgust', 'fearful',
       'happy', 'calm', 'calm', 'fearful', 'fearful', 'disgust', 'calm',
       'fearful', 'calm', 'calm', 'calm', 'fearful', 'happy', 'calm',
       'fearful', 'calm', 'fearful', 'fearful', 'calm', 'happy', 'calm',
       'happy

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 65.64%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.69291339, 0.55      , 0.75247525, 0.58536585])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,disgust,disgust
1,fearful,fearful
2,calm,calm
3,disgust,calm
4,calm,calm
5,calm,calm
6,happy,calm
7,disgust,disgust
8,fearful,fearful
9,fearful,fearful


In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab Notebooks/RAVDESS_Emotional_speech_audio/happy (1).wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')

In [ ]:
feature

NameError: ignored

In [ ]:
/content/drive/MyDrive/Colab Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav